this is for identifying all NaNs, and fill them again with the values acquired from the requests.

r5 result NA handling.

Pick out the nan pairs, and query the travel time separately, to fill up the nans.


Step 1: 
Read in the r5 matrix, and get the OD pairs:


In [2]:
#from r5py import TransportNetwork, TravelTimeMatrixComputer
#,DetailedItinerariesComputer


In [1]:
import numpy as np
import r5py
import pandas as pd
from scipy.stats import ttest_rel, wilcoxon
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import geopandas as gpd
from shapely.geometry import Point
from r5py import TravelTimeMatrixComputer
from r5py import TransportNetwork

import sys


set up R5 running parameter

In [2]:
sys.argv.append(["--max-memory", "12G"])

In [3]:
r5TTM = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/greater_london_od_mat_all/withNaN/r5_od_mat.csv', encoding='latin1')

In [4]:
r5TTM.shape

(1002, 1003)

In [5]:
r5TTM.head()

,originID,E02000001,E02000002,E02000003,E02000004,E02000005,E02000007,E02000008,E02000009,E02000010,...,E02007083,E02007084,E02007108,E02007109,E02007110,E02007111,E02007112,E02007113,E02007114,E02007115
0,E02000001,0.0,37.0,34.0,35.0,32.0,33.0,30.0,29.0,30.0,...,22.0,23.0,12.0,12.0,9.0,7.0,19.0,19.0,20.0,10.0
1,E02000002,37.0,0.0,9.0,13.0,12.0,14.0,13.0,12.0,16.0,...,51.0,52.0,29.0,31.0,34.0,34.0,26.0,26.0,27.0,43.0
2,E02000003,33.0,5.0,0.0,9.0,4.0,6.0,6.0,6.0,8.0,...,46.0,48.0,26.0,28.0,31.0,31.0,21.0,21.0,22.0,41.0
3,E02000004,35.0,15.0,9.0,1.0,8.0,5.0,8.0,9.0,7.0,...,49.0,50.0,34.0,36.0,39.0,38.0,24.0,24.0,25.0,45.0
4,E02000005,31.0,10.0,4.0,7.0,0.0,4.0,3.0,5.0,5.0,...,45.0,46.0,30.0,32.0,35.0,34.0,20.0,19.0,21.0,41.0


In [6]:
r5TTM.set_index('originID', inplace=True)

In [7]:
# Set diagonal elements to 0
for i in range(len(r5TTM)):
    r5TTM.iat[i, i] = 0

In [8]:
# count total nans:
print(r5TTM.isnull().sum().sum())

873


Step 2 after handling diagonal values, look in to nan, we have 870+ nans now.

In [9]:
nan_indices = []
for i in range(len(r5TTM)):
    for j in range(len(r5TTM.columns)):
        if pd.isnull(r5TTM.iat[i, j]):
            nan_indices.append((r5TTM.index[i], r5TTM.columns[j]))

In [10]:
nan_indices

[('E02000372', 'E02000001'),
 ('E02000372', 'E02000002'),
 ('E02000372', 'E02000003'),
 ('E02000372', 'E02000004'),
 ('E02000372', 'E02000005'),
 ('E02000372', 'E02000007'),
 ('E02000372', 'E02000008'),
 ('E02000372', 'E02000009'),
 ('E02000372', 'E02000010'),
 ('E02000372', 'E02000011'),
 ('E02000372', 'E02000012'),
 ('E02000372', 'E02000013'),
 ('E02000372', 'E02000014'),
 ('E02000372', 'E02000015'),
 ('E02000372', 'E02000016'),
 ('E02000372', 'E02000017'),
 ('E02000372', 'E02000018'),
 ('E02000372', 'E02000019'),
 ('E02000372', 'E02000020'),
 ('E02000372', 'E02000021'),
 ('E02000372', 'E02000022'),
 ('E02000372', 'E02000023'),
 ('E02000372', 'E02000024'),
 ('E02000372', 'E02000025'),
 ('E02000372', 'E02000026'),
 ('E02000372', 'E02000027'),
 ('E02000372', 'E02000028'),
 ('E02000372', 'E02000029'),
 ('E02000372', 'E02000030'),
 ('E02000372', 'E02000031'),
 ('E02000372', 'E02000032'),
 ('E02000372', 'E02000033'),
 ('E02000372', 'E02000034'),
 ('E02000372', 'E02000035'),
 ('E02000372',

Step 3 use the ID pairs to get coord pairs:

In [11]:
# Set up the R5 network
#osm_file = r'/mnt/d/Downloads/greater-london-latest.osm(1).pbf'
#r5_network = r5py.TransportNetwork(osm_file)

msoa_london = pd.read_csv(r'/mnt/d/University College London/Chen, Huanfa - Dissertation_Mengyu_Ding/Data/msoa/processed/gld_msoa_centroids.csv', encoding='latin1')
# see if the column msoa21cd is unique:
msoa_london['msoa21cd'].nunique() == msoa_london.shape[0]

# follow this format: [{"lat": ..., "lng": ...}, ...]

# new dict origin: 1002 maoas
all_MSOA_coord_dict = {}

# for reference dict, read the msoa_london, use msoa21cd as key, lat and lon as value:
for i in range(msoa_london.shape[0]):
    msoaID = msoa_london.iloc[i, 3]
    lat = msoa_london.iloc[i, 8]
    lon = msoa_london.iloc[i, 9]
    #origin_coord.append({"lat": lat, "lng": lon})
    all_MSOA_coord_dict[msoaID] = {"lat": lat, "lng": lon}
all_MSOA_coord_df = pd.DataFrame(all_MSOA_coord_dict).T
all_MSOA_coord_df = all_MSOA_coord_df.reset_index()

all_MSOA_coord_df.columns = ['id', 'lat', 'lon']

all_MSOA_coord_df['geometry'] = all_MSOA_coord_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
all_MSOA_gdf = gpd.GeoDataFrame(all_MSOA_coord_df, geometry='geometry')
all_MSOA_gdf.set_crs(epsg=4326, inplace=True)  

,id,lat,lon,geometry
0,E02000318,51.488265,0.095618,POINT (0.09562 51.48827)
1,E02000388,51.483157,-0.198445,POINT (-0.19845 51.48316)
2,E02000652,51.415041,-0.138046,POINT (-0.13805 51.41504)
3,E02000963,51.529079,-0.209371,POINT (-0.20937 51.52908)
4,E02000651,51.419153,-0.125942,POINT (-0.12594 51.41915)
...,...,...,...,...
997,E02000481,51.558124,0.293234,POINT (0.29323 51.55812)
998,E02000885,51.512556,-0.053381,POINT (-0.05338 51.51256)
999,E02000674,51.441193,-0.001961,POINT (-0.00196 51.44119)
1000,E02000975,51.514949,-0.179854,POINT (-0.17985 51.51495)


The blow is extra code to get the distance:

In [12]:
all_MSOA_gdf.loc[all_MSOA_gdf['id'] == 'E02000372', 'lat'] = 51.51645884386242
all_MSOA_gdf.loc[all_MSOA_gdf['id'] == 'E02000372', 'lon'] = -0.23745145532614184

In [13]:
# Set up the R5 network
osm_file = r'/mnt/d/Downloads/greater-london-latest.osm.pbf'
r5_network = r5py.TransportNetwork(osm_file)

In [14]:
from r5py import DetailedItinerariesComputer

In [23]:
from r5py import DetailedItinerariesComputer
from r5py import TransportMode

travel_details_matrix_computer = DetailedItinerariesComputer(
    r5_network,
    origins=all_MSOA_gdf,
    destinations=all_MSOA_gdf,
    departure=datetime.datetime(2024, 7, 23, 7, 30),
    transport_modes=[TransportMode.CAR],
    
    # With following attempts to snap all origin and destination points to the transport network before routing
    snap_to_network=True
)

# Compute the travel time matrix
travel_details_all = travel_details_matrix_computer.compute_travel_details()


java.util.NoSuchElementException: java.util.NoSuchElementException

Turned out running R5 with distance calculated can be a bit difficult:

1. Needs GTFS file to run the DetailedItinerariesComputer function.
2. Random NaNs that is not fillable.

A mixup here, travel detials all matrix is actually not a matrix, need to rename it properly. I'll save this one to the csv file as raw data for extracting the distance calculated by R5 first.

The above is an attempt to get distance

Step 4 Query the missing travel time with the same parameter:

In [15]:
origins_list = [x[0] for x in nan_indices]

In [16]:
dest_list = [x[1] for x in nan_indices]

The root cause of this problem:

MSOA centroid in a park too far from network,

Solution:
use the R5 "snap to network" method:

In [17]:
# Set up the R5 network
osm_file = r'/mnt/d/Downloads/greater-london-latest.osm.pbf'
r5_network = r5py.TransportNetwork(osm_file)

In [18]:
origin_gdf = all_MSOA_gdf[all_MSOA_gdf['id'].isin(origins_list)]
dest_gdf = all_MSOA_gdf[all_MSOA_gdf['id'].isin(dest_list)]


In [19]:
origin_gdf.head()

,id,lat,lon,geometry
702,E02000372,51.522036,-0.237708,POINT (-0.23771 51.52204)


In [21]:
dest_gdf.shape

(873, 4)

snap to network

In [22]:
#origin_gdf["snapped_geometry"] = r5_network.snap_to_network(origin_gdf["geometry"])
origin_gdf["snapped_geometry"] = r5_network.snap_to_network(origin_gdf["geometry"], 
                                                            radius=150, 
                                                            street_mode=r5py.TransportMode.CAR,
)

/home/mengyuwsl/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [26]:
# create a copy of origins gpf named snapped_origin_gdf, where geometry is snapped_geometry:
snapped_origin_gdf = origin_gdf.copy()

# modify the lat and lon column to match the value in snapped_geometry
snapped_origin_gdf['lat'] = snapped_origin_gdf['snapped_geometry'].apply(lambda x: x.y)
snapped_origin_gdf['lon'] = snapped_origin_gdf['snapped_geometry'].apply(lambda x: x.x)

# drop geometry column
snapped_origin_gdf.drop(columns=['geometry'], inplace=True)

# rename snapped_geometry to geometry
snapped_origin_gdf.rename(columns={'snapped_geometry': 'geometry'}, inplace=True)


In [27]:
snapped_origin_gdf.head()

,id,lat,lon,geometry
702,E02000372,51.52163,-0.239274,POINT (-0.23927 51.52163)


In [30]:
# change lat to 51.5198233902299 and lon to -0.23759461725201525
# 51.51645884386242, -0.23745145532614184
snapped_origin_gdf.loc[snapped_origin_gdf['id'] == 'E02000372', 'lat'] = 51.51645884386242
snapped_origin_gdf.loc[snapped_origin_gdf['id'] == 'E02000372', 'lon'] = -0.23745145532614184

# now change geometry to the new lat and lon
snapped_origin_gdf['geometry'] = snapped_origin_gdf.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

In [31]:
snapped_origin_gdf.head()

,id,lat,lon,geometry
702,E02000372,51.516459,-0.237451,POINT (-0.23745 51.51646)


In [25]:
# make a copy of all_MSOA_gdf:
all_msoa_gdf_copy= all_MSOA_gdf.copy()

# replace the id = 'E02000372' row with snapped geometry the value from snapped_origin_gdf:
all_msoa_gdf_copy.loc[all_msoa_gdf_copy['id'] == 'E02000372', 'geometry'] = snapped_origin_gdf['geometry'].values[0]
all_msoa_gdf_copy.loc[all_msoa_gdf_copy['id'] == 'E02000372', 'lat'] = snapped_origin_gdf['lat'].values[0]
all_msoa_gdf_copy.loc[all_msoa_gdf_copy['id'] == 'E02000372', 'lon'] = snapped_origin_gdf['lon'].values[0]

In [26]:
#check if the geometry is replaced:
all_msoa_gdf_copy.loc[all_msoa_gdf_copy['id'] == 'E02000372']

,id,lat,lon,geometry
702,E02000372,51.519823,-0.237595,POINT (-0.23759 51.51982)


now re-calculate our all-to-all again:

# this is the corret nan handling:

In [43]:
# Initialize the travel time matrix computer
ttm_computer = TravelTimeMatrixComputer(
    r5_network,
    origins=snapped_origin_gdf,
    destinations=dest_gdf,
    departure=datetime.datetime(2024, 7, 23, 7, 30),
    transport_modes=["CAR"]
    #snap_to_network = True
    
)

# Compute the travel time matrix
travel_time_nan_matrix = ttm_computer.compute_travel_times()

# Convert the travel time matrix to a DataFrame
travel_time_nan_df = pd.DataFrame(travel_time_nan_matrix)


In [44]:
# calcul total number of nans:
print(travel_time_nan_df.isnull().sum().sum())

0


In [45]:
travel_time_nan_df.head()

,from_id,to_id,travel_time
0,E02000372,E02000318,57
1,E02000372,E02000652,34
2,E02000372,E02000651,34
3,E02000372,E02000216,50
4,E02000372,E02000746,47


In [47]:
# make a copy:
r5TTM_copy = r5TTM.copy()

## now put the pairs back:

In [48]:
#	E02000372

# get the row from r5TTM_copy where index is E02000372:
r5TTM_copy.loc['E02000372']

E02000001   NaN
E02000002   NaN
E02000003   NaN
E02000004   NaN
E02000005   NaN
             ..
E02007111   NaN
E02007112   NaN
E02007113   NaN
E02007114   NaN
E02007115   NaN
Name: E02000372, Length: 1002, dtype: float64

In [49]:
# within this column if the column index matches the destID, replace the value with the value from travel_time_nan_df:
for idx, row in travel_time_nan_df.iterrows():
    r5_index = row['from_id']
    col_index = row['to_id']
    value = row['travel_time']
    # Only replace if the current value is NaN
    if pd.isna(r5TTM_copy.at[r5_index, col_index]):
        r5TTM_copy.at[r5_index, col_index] = value

In [109]:
for _, row in travel_time_nan_df.iterrows():
    from_id = row["from_id"]
    to_id = row["to_id"]
    travel_time = row["travel_time"]
    if from_id in r5TTM_copy.index and to_id in r5TTM_copy.columns:
        r5TTM_copy.loc[from_id, to_id] = travel_time

In [41]:
# count total number of nulls in r5TTM_copy:
print(r5TTM_copy.isnull().sum().sum())


0


In [50]:
# Save results to a CSV file:
r5TTM_copy.to_csv("ttm_r5py_nan_handled.csv", index=True)

In [51]:
#show column from_id == 'E02000372'
travel_time_nan_df[travel_time_nan_df['from_id'] == 'E02000372']


,from_id,to_id,travel_time
0,E02000372,E02000318,57
1,E02000372,E02000652,34
2,E02000372,E02000651,34
3,E02000372,E02000216,50
4,E02000372,E02000746,47
...,...,...,...
868,E02000372,E02000865,35
869,E02000372,E02000481,66
870,E02000372,E02000885,36
871,E02000372,E02000674,50


In [116]:
r5TTM_copy.loc['E02000372', 'E02000652']

34.0

In [79]:
origin_gdf

,id,lat,lon,geometry
702,E02000372,51.522036,-0.237708,POINT (-0.23771 51.52204)


In [80]:
dest_gdf.head()

,id,lat,lon,geometry
0,E02000318,51.488265,0.095618,POINT (0.09562 51.48827)
2,E02000652,51.415041,-0.138046,POINT (-0.13805 51.41504)
4,E02000651,51.419153,-0.125942,POINT (-0.12594 51.41915)
5,E02000216,51.369492,-0.077637,POINT (-0.07764 51.36949)
6,E02000746,51.516202,0.077463,POINT (0.07746 51.51620)


In [112]:
travel_time_nan_df

,from_id,to_id,travel_time
0,E02000372,E02000318,58
1,E02000372,E02000652,34
2,E02000372,E02000651,35
3,E02000372,E02000216,50
4,E02000372,E02000746,47
...,...,...,...
868,E02000372,E02000865,36
869,E02000372,E02000481,65
870,E02000372,E02000885,36
871,E02000372,E02000674,51


In [46]:
# calculate number of unique values in the travel time nan df column from_id:
travel_time_nan_df['from_id'].nunique()

1

In [82]:
from r5py import TransportNetwork, RegionalTask, TravelTimeMatrixComputer#, TravelDistanceMatrixComputer
# Define the origin and destination coordinates (latitude, longitude)
origin = Point(-0.237708,51.522036)  # Example: New York, Empire State Building
destination = (51.488265, 0.095618)  # Example: New York, One World Trade Center51.522036	-0.237708
origin_point = origin_gdf.geometry.iloc[0]

# Create a RegionalTask for the query
task = TravelTimeMatrixComputer(
    r5_network,
    origin=origin_gdf,  # Modified keyword argument from 'origins' to 'origin'
    destinations=dest_gdf,  # Modified keyword argument from 'destinations' to 'destination'
    departure=datetime.datetime(2024, 7, 23, 7, 30),  # Define departure time
    transport_modes=["CAR"],  # Mode of transportation
    
)

# Create a TravelTimeMatrixComputer to compute travel times
#computer = TravelTimeMatrixComputer(r5_network, task)

# Compute the travel times
#travel_times = computer.compute_travel_times()

travel_times = task.compute_travel_times()

# Extract and display the travel time for the query
travel_time = travel_times[0][0]
print(f"Travel time from origin to destination: {travel_time} minutes")

TypeError: r5py.r5.regional_task.RegionalTask() got multiple values for keyword argument 'origin'

In [ ]:
# Initialize the travel time matrix computer
ttm_computer = TravelTimeMatrixComputer(
    r5_network,
    origins=all_msoa_gdf_copy,
    destinations=all_msoa_gdf_copy,
    departure=datetime.datetime(2024, 7, 23, 7, 30),
    transport_modes=["CAR"]
)

# Compute the travel time matrix
travel_time_matrix = ttm_computer.compute_travel_times()

# Convert the travel time matrix to a DataFrame
travel_time_df = pd.DataFrame(travel_time_matrix)

# check the total number of nans:
print(travel_time_df.isnull().sum().sum())